# Gérer des données numériques dans un projet d’apprentissage supervisé

C’est bien connu, les machines adorent les nombres. Ils ont cela de commode qu’ils se prêtent mieux aux calculs que des symboles comme *pomme* ou *Pommes*. Dans un projet de *machine learning*, les données fournies aux algorithmes d’apprentissage devront toutes être représentées sous forme numérique et, avant d’aborder la manière de vectoriser des chaînes de caractère, il est plus sage de commencer par la manipulations des nombres, qu’ils soient entiers ou décimaux.

## Identifier une variable aléatoire quantitative

En statistiques, une variable aléatoire est l’une des caractéristiques d’une observation. Elle peut se représenter de manière rudimentaire sous forme de tableau à deux dimensions :

|Sexe|Taille|
|-|:-:|
|F|180|
|M|172|
|M|167|

Le tableau étant composé de trois lignes et de deux colonnes, il est réputé présenter deux caractéristiques pour trois observations dans une structure de dimensions $3\times 2$.

Sans se tromper, la variable aléatoire *Sexe* n’est pas de type numérique quand la variable *Taille*, elle, l’est.

Comment s’en assurer avec Python ? La propriété `dtypes` d’un *data frame* permet de s’en assurer rapidement :

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'gender': ['F', 'M', 'M'],
    'height': [180, 172, 167]
})

df.dtypes

La variable `height` est bien de type numérique. Pour autant, une variable aléatoire représentée sous forme numérique est-elle systématiquement quantitative, au sens statistique du terme ?

### Numérique ≠ quantitatif

Pour qu’une variable numérique soit considérée comme quantitative, elle est censée pouvoir exprimer une quantité. Après avoir ajouté les années de naissance des individus, la propriété `dtypes` signale que la variable `birth`, conformément à l’intuition, est bien de type numérique :

In [ ]:
df["birth"] = [1983, 2001, 2011]

df.dtypes

La variable `birth` est-elle quantitative ? Pour le déterminer facilement, il faut se poser la question de savoir si cela fait sens de cumuler les valeurs consignées. Quand les tailles des individus peuvent former une somme pour obtenir ensuite une moyenne, est-ce raisonnable d’additionner des années de naissance ? Le calcul de la moyenne arithmétique des années de naissance donne pour résultat : $1998,\overline{3}$. Est-il logique de poser ce genre de question ?

In [ ]:
df["birth"].mean()

Eh bien, en fait, oui. Si la moyenne ici n’est pas à proprement parler intéressante, il est légitime de se demander quelle est la médiane des années de naissance des individus interrogés, elle pourrait expliquer certains résultats. En revanche, il n’en serait pas de même des codes postaux de leur lieu d’habitation ou de leurs numéros de sécurité sociale.

### Quantitative discrète ou continue ?

La différence est encore parfois plus subtile entre les variables aléatoires quantitatives discrètes et continues. Dans l’exemple avec les années, la coutume est de considérer leur valeur comme une discrétion du temps qui, lui, est continu ; d’un autre côté, si le jeu de données comporte une représentation décimale des années, où $1998,55$ équivaudrait au 15 juin 1998, rien n’interdit de considérer qu’il existe alors une continuité.

Parfois, une distinction commode véhiculée par certaines sources consiste à considérer une donnée représentée par un élément de l’ensemble $\mathbb{Z}$ comme discrète, mais appliquer cette préconisation sans réflexion peut amener à des erreurs d’interprétation. L’âge, par exemple, est souvent noté sous forme d’entiers naturels. Il s’agit d’une convention : personne ne passe réellement de $x$ ans à $x+1$ ans sans vivre les intervalles, si ? Pour s’en assurer tout à fait, demandons-nous si le calcul de l’âge moyen des individus d’une enquête ferait sens.

En revanche, si au moment de la préparation des données de l’enquête, on établissait des classes d’âge (moins de 18 ans, plus de 35 ans etc.), la variable deviendrait discrète, et qualitative. D’autres données posent les mêmes difficultés, comme la taille, ou le poids, qui, comme elles sont exprimées avec une unité et ne peuvent prendre qu’une valeur isolée, sont cataloguées généralement comme valeurs discrètes. Pourtant, la taille et le poids **d’un individu** peuvent prendre, si mesurés précisément, n’importe quelle valeur dans un intervalle (p. ex. : de 0 à 300 cm) et exprimer ainsi une continuité.

Comment, alors, être sûr·es de faire le bon choix ? Dans le doute, une bonne option est de se reposer sur la représentation graphique de la variable en ballottage :
- Un diagrammes en barres pour une quantitative discrète ;
- un histogramme pour une quantitative continue.

Dans notre petit jeu de données, nous avons catalogué la variable *année* comme une quantitative discrète. Représentons-là avec un diagramme en barres, puis un histogramme :

![Représentation des années avec un diagramme en barres](./images/birth-barplot.png)

![Représentation des années avec un histogramme](./images/birth-histogram.png)

Quand le premier graphique parle de lui-même, le second peine à convaincre.

## La délicate question du pré-traitement des données

Un algorithme de *machine learning* est grandement dépendant de la qualité des données sur lesquelles il est entraîné. Pour cette raison, la phase de pré-traitement (*pre-processing*) est cruciale. Il s’agira de ne laisser aucune donnée manquante dans le jeu de données et d’harmoniser les grandeurs des variables numériques.

### La chasse aux données manquantes…

Comment repérer les données manquantes dans un *dataset* et, surtout, comment les gérer ? Pour une seule variable sans valeur, faut-il supprimer toute l’observation ? Et s’il est question de la remplacer, quelle valeur choisir ?

Commençons par charger le jeu de données sur le recensement des manchots de l’Antarctique et affichons un résumé du *data frame* obtenu :

In [ ]:
df = pd.read_csv("./data/penguin-census-numerical-features.csv")

df.info()

Il en ressort que les 344 observations ne sont pas toutes complètes : pour deux d’entre elles il manque les caractéristiques physiques.

#### Supprimer les observations avec données manquantes

La première stratégie consiste à supprimer les observations concernées. Après tout, deux observations sur 344 ne représente qu’une perte de 0,5 % de l’ensemble.

In [ ]:
data = df.dropna()

data.info()

Dans ce cas de figure, l’argument se tient, parce que les valeurs manquaient toutes pour les deux mêmes observations. Mais si elles avaient concerné des manchots différents, on aurait alors supprimé 8 observations, soit 2 % de l’ensemble.

#### Remplacer par une valeur

Plusieurs options se présentent : remplacer par des zéros, par une valeur fixe, par la moyenne, par la médiane ou encore par la valeur la plus représentée. Chacune de ces options a ses avantages et ses inconvénients.

*Scikit-Learn* dispose d’une classe `SimpleImputer` pour réaliser n’importe laquelle de ces options. Elle prend un paramètre `strategy`, dont les valeurs sont à choisir parmi : `mean` (option par défaut), `median`, `most_frequent`, `constant`. Si la stratégie `constant` est sélectionnée, il faut indiquer la valeur dans un paramètre `fill_value`.

Remplaçons dans un premier temps les valeurs manquantes par une valeur fixe, le zéro :

In [ ]:
from sklearn.impute import SimpleImputer

# copy of the data frame
data = pd.DataFrame.copy(df)
# new instance
imputer = SimpleImputer(strategy="constant", fill_value=0)
# fit the imputer to data
imputer.fit(data)
# create a matrice
X = imputer.transform(data)

# missing values for the 4th sample are now fixed to 0
X[3]

Remplaçons maintenant par la valeur médiane :

In [ ]:
# copy
data = pd.DataFrame.copy(df)
# only numerical features
data = data.drop(columns="species")
# an imputer with median strategy
imputer = SimpleImputer(strategy="median")
# shortcut for fit then transform
X = imputer.fit_transform(data)

# values for the 4th sample
X[3]

### … et aux données aberrantes

Les mêmes stratégies peuvent s’appliquer aux données aberrantes, les valeurs extrêmes pouvant affecter négativement certaines mesures. Il s’agit parfois d’un zéro surnuméraire ou du déplacement de la virgule dans la notation décimale d’une quantité. La moyenne arithmétique est par exemple est très sensible à ces erreurs. Et il en va de même des algorithmes d’apprentissage automatique.

### Mise à l’échelle

Les données d’une observation font rarement toutes référence à une échelle commune. L’âge d’un individu sera compris entre 0 et 100, sa taille entre 0 et 200, son score de satisfaction entre 0 et 10, la numération de ses globules rouges entre 3 000 000 et 6 000 000 etc. Il faut savoir que les algorithmes d’apprentissage sont sensibles à la différence entre les grandeurs et fourniront des prédictions de mauvaise qualité si certaines variables sont réparties dans un espace bien plus vaste que les autres.

La mise à l’échelle consiste alors à réduire leur variance ou leur valeur absolue. Plusieurs méthodes existent et, parmi les plus utilisées, citons la standardisation et la normalisation.

Avant toutes choses, récupérons une variable descriptive des manchots, la longueur du bec, en remplaçant les valeurs manquantes :

In [ ]:
bill_length_median = df["bill_length_mm"].median()
bill_length = df["bill_length_mm"].fillna(bill_length_median)

#### La standardisation

La standardisation (*Z score normalization*) consiste à centrer la variable autour de 0 de telle manière que son écart-type soit égal à 1. La formule donne avec $\mu$ pour la moyenne et $\sigma$ pour l’écart-type :

$$f(x) = \frac{x − \mu}{\sigma}$$

Avant de centrer-réduire la variable *bill_length_mm*, l’affichage de sa moyenne et de son écart-type donne 43,92 pour la première et 5,44 pour la seconde.

In [ ]:
print(
    f"Mean value: { bill_length.mean().round(2) }",
    f"Standard deviation: { bill_length.std().round(2) }",
    sep="\n"
)

Concrètement, l’opération de standardisation va d’abord soustraire la moyenne puis diviser ensuite le résultat par l’écart-type. Si l’on effectue ce calcul à la main, on obtient bien une moyenne à 0 et un écart-type de 1 :

In [ ]:
bill_length_scaled = list()

[
    bill_length_scaled.append(
        (n - bill_length.mean()) / bill_length.std()
    )
    for n in bill_length
]
bill_length_scaled = pd.Series( (value for value in bill_length_scaled) )

print(
    f"Mean value: { bill_length_scaled.mean().round(2) }",
    f"Standard deviation: { bill_length_scaled.std().round(2) }",
    sep="\n"
)

Il existe heureusement une classe `StandardScaler` dans *Scikit-Learn* pour effectuer l’opération plus simplement :

In [ ]:
from sklearn.preprocessing import StandardScaler

# standard scaler
scaler = StandardScaler()

# reshape Serie to match 2d array
bill_length_scaled = scaler.fit_transform(bill_length.values.reshape(-1, 1))

print(
    f"Mean value: { bill_length_scaled.mean().round(2) }",
    f"Standard deviation: { bill_length_scaled.std().round(2) }",
    sep="\n"
)

#### La normalisation

Plus simple à appréhender, la normalisation Min-Max (*Min-Max normalization*) est une méthode qui va soustraire à chaque valeur la minimale puis la diviser ensuite par l’écart maximal de la série. Comme la formule est basée sur les extrêmes, elle est particulièrement sensible aux données aberrantes.

$$f(x) = \frac{x − min(x)}{max(x) − min(x)}$$

Le résultat n’est plus une variable centrée réduite, mais une variable dont les valeurs seront contenues dans un intervalle $[0, 1]$.

En reprenant l’exemple précédent sur la longueur du bec des manchots, nous appliquons cette fois-ci une classe `MinMaxScaler` :

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
bill_length_scaled = scaler.fit_transform(bill_length.values.reshape(-1, 1))

bill_length_scaled[:5]